<p style="text-align:center;">
<img src="figures/NCBI-Logo.png" alt="NCBI_database" width="200" class="center"/>
</p>
<p style="text-align:center;">
<a href="https://www.ncbi.nlm.nih.gov/">https://www.ncbi.nlm.nih.gov/</a> 
</p>

### Objetives
1. Find appropiate BLAST program
2. Enter query sequence
3. Select database to search
4. Run and analyze the output
5. Use Smith-Waterman algorithm to corroborate NCBI BLAST findings
6. Assess the significance of Smith-Waterman’s alignment score.

### Use Smith-Waterman algorithm to corroborate NCBI BLAST findings

In [1]:
import requests
from tqdm import tqdm

def download(url: str, fname: str):
    resp = requests.get(url, stream=True)
    total = int(resp.headers.get('content-length', 0))
    with open(fname, 'wb') as file, tqdm(
        desc=fname,
        total=total,
        unit='iB',
        unit_scale=True,
        unit_divisor=1024,
    ) as bar:
        for data in resp.iter_content(chunk_size=1024):
            size = file.write(data)
            bar.update(size)

In [2]:
download("http://hgdownload.soe.ucsc.edu/goldenPath/hg38/chromosomes/chr13.fa.gz", "../chrs/chr13.fa.gz")

../chrs/chr13.fa.gz: 100%|████████████████████████████████████████████████████████████████████████| 30.4M/30.4M [23:46<00:00, 22.3kiB/s]


In [ ]:
import gzip

with gzip.open("../chrs/chr13.fa.gz", "rt") as handle:
    for chr13 in SeqIO.parse(handle, "fasta"):
        print(len(chr13.seq))

In [ ]:
chr13_sub = chr13.seq[32315508:32400268]

In [ ]:
from Bio import SeqIO
with open("unknown_sequence.fasta", "rt") as handle:
    for unknown in SeqIO.parse(handle, "fasta"):
        print(len(unknown.seq))

In [ ]:
from Bio import Align

aligner = Align.PairwiseAligner()
aligner.match_score = 1
aligner.mismatch_score = -2
aligner.open_gap_score = -2
aligner.extend_gap_score = -2
aligner.mode = "local"

In [ ]:
alignments = aligner.align(chr13_sub, sequence.seq)

In [ ]:
def nucolored(seq):
    bcolors = {
        'A': '\033[92m',
        'C': '\033[94m',
        'G': '\033[93m',
        'T': '\033[91m',
        'U': '\033[91m',
        'reset': '\033[0;0m'
    }
    tmp_seq = ""
    for n in seq:
        if n in bcolors:
            tmp_seq += bcolors[n] + n
        else:
            tmp_seq += bcolors['reset'] + n
    return tmp_seq + '\033[0;0m'

def print_local_alignment(alignment):
    seq1 = alignment.target
    seq2 = alignment.query
    aligned_seq1 = ""
    aligned_seq2 = ""
    pattern = ""
    paths = alignment.path
    start1, start2 = paths[0]
    for end1, end2 in paths[1:]:
        if end1 == start1:
            gap = end2 - start2
            aligned_seq1 += "-" * gap
            aligned_seq2 += seq2[start2:end2]
            pattern += "-" * gap
        elif end2 == start2:
            gap = end1 - start1
            aligned_seq1 += seq1[start1:end1]
            aligned_seq2 += "-" * gap
            pattern += "-" * gap
        else:
            s1 = seq1[start1:end1]
            s2 = seq2[start2:end2]
            if len(s1) != len(s2):
                raise ValueError("Unequal step sizes in alignment")
            aligned_seq1 += s1
            aligned_seq2 += s2
            for c1, c2 in zip(s1, s2):
                if c1 == c2:
                    pattern += "|"
                else:
                    pattern += "."
        start1 = end1
        start2 = end2
    print(f"Score: {alignment.score}")
    print(f"Target {paths[0][0]:6} {nucolored(aligned_seq1)}")
    print(f"{' '*14}{pattern}")
    print(f"Query  {paths[0][1]:6} {nucolored(aligned_seq2)}")
    
from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

In [ ]:
print_local_alignment(alignments[0])

In [ ]:
scores = random_scores(1000, unknown.seq)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# getting data of the histogram
counts, bins_counts = np.histogram(scores, bins=10)
plt.stairs(counts, bins_counts, fill=True)

In [ ]:
mean = np.mean(counts)
std = np.std(counts)

lamb = 1.2825/std
khat = np.exp(lamb*mean - 0.5772)

p_value = 1-np.exp(-khat*np.exp(-lamb*1723))
e_value = khat*np.exp(-lamb*1723)

print(f"p value : {p_value}")
print(f"e value : {e_value}")